<a href="https://colab.research.google.com/github/jefflance/tp_note_python_240321/blob/main/LANCE_TP_SQLite_et_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP noté Bases de données

## Utilisation de sqlite3 avec colab

Dans ce TP nous allons utiliser le package python sqlite3. En association avec une plateforme comme Google Colab permettant d'héberger un notebook en ligne, il est extrêmement simple de créer une base de donnée, de la partager et de la modifier.

**Objectifs**: Créer une base de donnée de toute pièce, créer une base à partir de données existentes et effectuer des modification sur les tables.

## Préambule

Nous allons commencer par importer les package nécessaires et créer une base vierge et établir une connection avec celle-ci. Une description détatillée du package SQLite3 peut être trouvée [ici](https://docs.python.org/3/library/sqlite3.html) 

In [1]:
import sqlite3
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
con = sqlite3.connect('bdd.db') # connection avec la base bdd

## Création de tables vierges

Une fois la connection établie, il est possible d'instancier un curseur dans la base. Cet objet permet d' "éxécuter" des commandes SQL directement dans celle-ci via la méthode ```execute```. Il est possible enfin de sauvegarder les changement en appelant la méthode ```comit``` de la conection. Dans cet exemple, nous créons la table ```eleve(nom:str, prenom: str, date:date)``` grâce à la commande SQL ```CREATE TABLE``` .

In [ ]:
#@title
cur = con.cursor()

# Create table
cur.execute('''CREATE TABLE eleve
               (nom str, prenom str, date date)''')

On peut ajouter un élève dans la table grâce à la commande SQL *INSERT*

In [ ]:
#@title
# Insert a row of data
cur.execute("INSERT INTO eleve VALUES ('Michel','Jean','2006-03-28')")


Enfin on peut sauvegarder les changements

In [ ]:
# Save (commit) the changes
con.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
con.close()

### Exercice

Créer une nouvelle base de données correspondant à la base des voyageurs réduite à deux tables :

* ```Voyageur (**idVoyageur**, nom, prénom, ville, région)```
* ```Séjour (**idSéjour**, *idVoyageur*, codeLogement, début, fin)```

Les commandes SQL à connaitre sont


```
CREATE TABLE *nom_de_latable* (
    *nom_attribut 1* *type ex:*int,
    *nom_attribut 2* *type ex:*str,
    ...,
    PRIMARY KEY (*nom_attribut n*),
    FOREIGN KEY (*nom_attribut m*) REFERENCES *Nom d'une autre table*(*attribut dans cette table*));
```
 Une fois les tables crées, ajouter quelques lignes dans les deux tables :



```
INSERT INTO table_name
VALUES (value1, value2, value3, ...); 
```




In [2]:
cur = con.cursor()

# Create table Voyageur
cur.execute('''
  CREATE TABLE Voyageur (
    idVoyageur INTEGER PRIMARY KEY,
    nom TEXT NOT NULL, prénom TEXT NOT NULL,
    ville TEXT, région TEXT
  )
''')

# Create table Séjour
cur.execute('''
  CREATE TABLE Séjour (
    idSéjour INTEGER PRIMARY KEY,
    idVoyageur INTEGER NOT NULL,
    codeLogement INTEGER NOT NULL,
    début INTEGER, fin INTEGER,
    FOREIGN KEY (idVoyageur) REFERENCES Voyageur(idVoyageur)
  )
''')

In [3]:
# Add lines
cur.execute("INSERT INTO Voyageur VALUES (1, 'Dodouce', 'Marin', 'Paris', 'Île-de-France')")
cur.execute("INSERT INTO Voyageur VALUES (2, 'Pierre', 'Jean', 'Aubervilliers', 'Île-de-France')")
cur.execute("INSERT INTO Voyageur VALUES (3, 'Sticqué', 'Sophie', 'Meaux', 'Île-de-France')")
cur.execute("INSERT INTO Voyageur VALUES (4, 'Aimar', 'Jean', 'Provins', 'Île-de-France')")
cur.execute("INSERT INTO Voyageur VALUES (5, 'Coptaire', 'Eli', 'Montpellier', 'Occitanie')")


In [4]:
# Save (commit) the changes
con.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
con.close()

## Création d'une table à partir de données existentes

Télécharger le fichier .csv AABA donnant les valeurs de cours d'actions [ici](http://cedric.cnam.fr/~rambourc/AABA_2006-01-01_to_2018-01-01.csv). Nous alllons le charger et le parser automatiquement avec pandas

In [5]:
!wget http://cedric.cnam.fr/~rambourc/AABA_2006-01-01_to_2018-01-01.csv

--2021-03-25 20:28:01--  http://cedric.cnam.fr/~rambourc/AABA_2006-01-01_to_2018-01-01.csv
Resolving cedric.cnam.fr (cedric.cnam.fr)... 163.173.128.10
Connecting to cedric.cnam.fr (cedric.cnam.fr)|163.173.128.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 187560 (183K) [text/x-comma-separated-values]
Saving to: ‘AABA_2006-01-01_to_2018-01-01.csv’

AABA_2006-01-01_to_ 100%[===================>] 183.16K   434KB/s    in 0.4s    

2021-03-25 20:28:02 (434 KB/s) - ‘AABA_2006-01-01_to_2018-01-01.csv’ saved [187560/187560]



In [6]:
import pandas as pd
import numpy as np
dataset = pd.read_csv('AABA_2006-01-01_to_2018-01-01.csv', index_col='Date', parse_dates=['Date'])
dataset.head()

,Open,High,Low,Close,Volume,Name
Date,,,,,,
2006-01-03,39.69,41.22,38.79,40.91,24232729,AABA
2006-01-04,41.22,41.90,40.77,40.97,20553479,AABA
2006-01-05,40.93,41.73,40.85,41.53,12829610,AABA
2006-01-06,42.88,43.57,42.80,43.21,29422828,AABA
2006-01-09,43.10,43.66,42.82,43.42,16268338,AABA


### Exercice

Créer une nouvele table dans votre base de données intitulée *cours* avec le schéma suivant :

* ```Cours (**date**:date, open:float, high:float, volume:float)```

In [7]:
con = sqlite3.connect('bdd.db') # connection avec la base bdd
cur = con.cursor()

cur.execute('''
  CREATE TABLE Cours (
    date DATE PRIMARY KEY,
    open REAL, high REAL,
    volume REAL
  )
''') # Votre code à la place des ...

Ajouter tous les éléments du fichier dans votre base de donnée grâce à une boucle *for*. Vous pouvez ajouter des variable dans vos requêtes SQL de la façon suivante



```
cur.execute('INSERT INTO Cours VALUES (?,?,?,?)',a,b,c,d)
```
Ici, la valeur des variable *a,b,c* et *d* sera mise à la place des *?* dans la requête.

**Attention**: vous devez transformer les date (premier élément) en string avec la fonction ```str()```


In [8]:
for date, row in dataset.iterrows():
  cur.execute('INSERT INTO Cours VALUES (?, ?, ?, ?)', (str(date), row['Open'], row['High'], row['Volume']))

con.commit()

Vous pouvez vérifier ce que contient votre table en affichant les lignes

In [9]:
for row in cur.execute('SELECT * FROM Cours ORDER BY date'):
  print(row)

('2006-01-03 00:00:00', 39.69, 41.22, 24232729.0)
('2006-01-04 00:00:00', 41.22, 41.9, 20553479.0)
('2006-01-05 00:00:00', 40.93, 41.73, 12829610.0)
('2006-01-06 00:00:00', 42.88, 43.57, 29422828.0)
('2006-01-09 00:00:00', 43.1, 43.66, 16268338.0)
('2006-01-10 00:00:00', 42.96, 43.34, 16288580.0)
('2006-01-11 00:00:00', 42.19, 42.31, 26192772.0)
('2006-01-12 00:00:00', 41.92, 41.99, 18921686.0)
('2006-01-13 00:00:00', 41.0, 41.08, 30966185.0)
('2006-01-17 00:00:00', 39.09, 40.39, 42429911.0)
('2006-01-18 00:00:00', 35.01, 36.16, 118670393.0)
('2006-01-19 00:00:00', 35.82, 35.84, 60918436.0)
('2006-01-20 00:00:00', 34.44, 34.66, 57681688.0)
('2006-01-23 00:00:00', 34.22, 34.4, 30894265.0)
('2006-01-24 00:00:00', 34.55, 35.2, 31686324.0)
('2006-01-25 00:00:00', 35.43, 35.48, 23786001.0)
('2006-01-26 00:00:00', 34.94, 35.25, 28480917.0)
('2006-01-27 00:00:00', 35.26, 35.27, 24325960.0)
('2006-01-30 00:00:00', 35.06, 35.23, 29030615.0)
('2006-01-31 00:00:00', 35.2, 35.2, 36538014.0)
('2006

## Exercice

A présent vous pouvez extraire une information particulière de la table cours en utilisant l'opérateur ```WHERE```.
Récupérez dans la table toutes les valeurs du mois de janvier 2006. Pour ça utilisez la condition 

``` WHERE *attribut* LIKE "année%mois%"```

Le symbole ```%``` est interprété comme n'importe quelle chaine de caractère

In [10]:
# Mettez votre requête SQL à la place des ...
rows = cur.execute('''
  SELECT * FROM Cours
  WHERE date LIKE "2006%01%"
  ORDER BY date
''')

for row in rows:
  print(row)


con.close()

('2006-01-03 00:00:00', 39.69, 41.22, 24232729.0)
('2006-01-04 00:00:00', 41.22, 41.9, 20553479.0)
('2006-01-05 00:00:00', 40.93, 41.73, 12829610.0)
('2006-01-06 00:00:00', 42.88, 43.57, 29422828.0)
('2006-01-09 00:00:00', 43.1, 43.66, 16268338.0)
('2006-01-10 00:00:00', 42.96, 43.34, 16288580.0)
('2006-01-11 00:00:00', 42.19, 42.31, 26192772.0)
('2006-01-12 00:00:00', 41.92, 41.99, 18921686.0)
('2006-01-13 00:00:00', 41.0, 41.08, 30966185.0)
('2006-01-17 00:00:00', 39.09, 40.39, 42429911.0)
('2006-01-18 00:00:00', 35.01, 36.16, 118670393.0)
('2006-01-19 00:00:00', 35.82, 35.84, 60918436.0)
('2006-01-20 00:00:00', 34.44, 34.66, 57681688.0)
('2006-01-23 00:00:00', 34.22, 34.4, 30894265.0)
('2006-01-24 00:00:00', 34.55, 35.2, 31686324.0)
('2006-01-25 00:00:00', 35.43, 35.48, 23786001.0)
('2006-01-26 00:00:00', 34.94, 35.25, 28480917.0)
('2006-01-27 00:00:00', 35.26, 35.27, 24325960.0)
('2006-01-30 00:00:00', 35.06, 35.23, 29030615.0)
('2006-01-31 00:00:00', 35.2, 35.2, 36538014.0)
('2006